In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import LineString, box, Point
from shapely import wkt
import numpy as np
import swifter

from generator import RoadNetwork


In [ ]:
# normal train porto dataset
df = pd.read_csv("../datasets/trajectories/Porto/train.csv")
# indices = np.random.choice(df.shape[0], int(df.shape[0]/10))
# df = df.iloc[indices, :].reset_index()
df['POLYLINE'] = df['POLYLINE'].swifter.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry="POLYLINE")
gdf["coords"] = gdf["POLYLINE"].swifter.apply(lambda x: list(x.coords))

In [ ]:
from ast import literal_eval
# porto clipped dataset with min 10 gps points per trajectory
df_c = pd.read_csv("../datasets/trajectories/Porto/clipped_porto_10pmin_2mil.csv")
df_c['POLYLINE'] = df_c['POLYLINE'].swifter.apply(wkt.loads)
gdf_c = gpd.GeoDataFrame(df_c, crs='epsg:4326', geometry="POLYLINE")
gdf_c["coords"] = gdf_c["coords"].swifter.apply(literal_eval)

In [ ]:
print(gdf["coords"].str.len().sum(), gdf_c["coords"].str.len().sum())

In [ ]:
# Folium Heat Map
from folium import plugins
from folium.plugins import HeatMap
from collections import defaultdict
import branca.colormap

map_porto = folium.Map(location=[41.18049, -8.64599],
                    zoom_start = 8)

coords = gdf.iloc[:, -1].values
print(coords.shape)
# cut down to 1/2 random choice, since full data is too big
# indices = np.random.choice(coords.shape[0], int(coords.shape[0]/10))
# reduced_coords = coords[indices]
# print(reduced_coords.shape)
coords = np.concatenate(coords)
print(coords.shape)

heat_data = [(c[1], c[0]) for c in coords]

steps=20
colormap = branca.colormap.linear.viridis.scale(0, 1).to_step(steps)
colormap.caption = 'Density of GPS points with 10m radius'
gradient_map=defaultdict(dict)
for i in range(steps):
    gradient_map[1/steps*i] = colormap.rgb_hex_str(1/steps*i)
colormap.add_to(map_porto)

HeatMap(heat_data, radius=10, gradient=gradient_map).add_to(map_porto)


# Display the map
#map_porto.save("heatmap_gps_points_porto.html")
map_porto

In [ ]:
"""
Analysis of Road Segment mapping generated by fmm
"""
# visualize trajectories that have outlier speed data - one with negative speed and one with inf speed
# negative speeds = trajectories where the vehicle is standing
df = pd.read_csv("../datasets/trajectories/Porto/road-segment-mapping.csv", sep=";")
df.dropna(inplace=True)
df["speed"] = df["speed"].swifter.apply(literal_eval)
df["speed_mean"] = df["speed"].swifter.apply(np.mean)

In [ ]:
# generate samples
sample_neg = df[df["speed_mean"] < 0].iloc[3]
sample_inf = df[df["speed_mean"] > 1e1].iloc[1]
sample_neg["mgeom"] = wkt.loads(sample_neg["mgeom"])
sample_inf["mgeom"] = wkt.loads(sample_inf["mgeom"])

sample_inf["speed_km"] = np.around(np.array(sample_inf["speed"]) * 111000 * 3.6, decimals=1)
step = int(len(list(sample_inf["mgeom"].coords)) / len(sample_inf["speed"]))
sample_inf

In [ ]:
# plot supspecious examples
m = folium.Map(
    location=[41.1372482, -8.689151],
    zoom_start=13,
    tiles="cartodbdark_matter"
)

data = [(c[1], c[0]) for c in list(sample_inf["mgeom"].coords)]
# d2 = [(c[1], c[0]) for c in list(so["geometry"].coords)]
#speed = df_an["speed"][3]
folium.PolyLine(data, color="green", weight=2.5, opacity=0.5).add_to(m)
#folium.PolyLine(d2, color="red", weight=2.5, opacity=1).add_to(m)
for i, speed in enumerate(sample_inf["speed_km"]):
    folium.Marker(
        location=data[i*7],
        popup=str(speed),
        icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: white">{str(speed)} km/h</div>""")
    ).add_to(m)

m

In [ ]:
network = RoadNetwork()
network.load("../osm_data/porto")

In [ ]:
node = network.gdf_edges[(network.gdf_edges["length"]>800) & (network.gdf_edges["osmid"].str.len() < 12)].iloc[-1]
print(node.dropna().drop(["geometry", "highway_enc", "fid"]))

In [ ]:
# plot osm examples of way and node for prelim section
m = folium.Map(
    location=[41.1488008, -8.6121386],
    zoom_start=13,
    tiles="cartodbpositron"
)
# html = f'''
# id: {node.name}<br>
# latitude: {node.y}<br>
# longitude: {node.x}<br>
# highway: {node.highway}<br>
# '''

# iframe = folium.IFrame(html,
#                        width=100,
#                        height=100)

# popup = folium.Popup(iframe,
#                     show=True,
#                     max_width=100)

# folium.CircleMarker(location=[node.y, node.x],
#                         radius=5,
#                         #fill=True,
#                         #fill_opacity=1,
#                         color="red",
#                         # fill_color="red",
#                         weight=5).add_to(m)

data = [(c[1], c[0]) for c in list(node["geometry"].coords)]
# d2 = [(c[1], c[0]) for c in list(so["geometry"].coords)]
#speed = df_an["speed"][3]
folium.PolyLine(data, color="red", weight=5, opacity=0.8).add_to(m)

m

In [6]:
"""
Generate Sample Image for trajectorie that goes out of network bounds and enters later again (For thesis visualization)
"""

df = pd.read_csv("../datasets/trajectories/Porto/mapped.csv")
df['POLYLINE'] = df['POLYLINE'].swifter.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry="POLYLINE")
gdf["coords"] = gdf["POLYLINE"].swifter.apply(lambda x: list(x.coords))
network = RoadNetwork()
network.load("../osm_data/porto")

Dask Apply:   0%|          | 0/32 [00:00<?, ?it/s]

In [34]:
xmin, ymin, xmax, ymax = network.bounds
out_of_bounds_traj = gdf.cx[xmax:, ymax:]

In [221]:
# sample = out_of_bounds_traj.iloc[np.random.randint(0, out_of_bounds_traj.shape[0])]
sample = out_of_bounds_traj[out_of_bounds_traj["Unnamed: 0"]==1155207]

m = folium.Map(
    location=[41.1372482, -8.689151],
    zoom_start=11,
    tiles="cartodbdark_matter"
)
# filter points 
in_data, out_data = [[], []], []
bbox = box(*network.bounds)
found_out = False
for c in sample.coords.values[0]:
    point = Point(c[0], c[1])
    if point.within(bbox):
        index = 0 if not found_out else 1
        in_data[index].append((c[1], c[0]))
    else:
        found_out = True
        out_data.append((c[1], c[0]))
data = [(c[1], c[0]) for c in sample.coords.values[0]]
folium.Rectangle([(ymin, xmin), (ymax, xmax)], color="#FE4A49", weight=1.5, opacity=1).add_to(m)
for ind in in_data:
    folium.PolyLine(ind, color="#c9ffe5", weight=2.5, opacity=1).add_to(m)
folium.PolyLine(out_data, color="#175676", weight=2.5, opacity=1).add_to(m)
folium.PolyLine([in_data[0][-1], out_data[0]], color="#c9ffe5", weight=2.5, opacity=1).add_to(m)
folium.PolyLine([in_data[1][0], out_data[-1]], color="#c9ffe5", weight=2.5, opacity=1).add_to(m)
# folium.PolyLine(d2, color="red", weight=2.5, opacity=1).add_to(m)
# 728703 169340 211514 1155207

m